# Outline
0. Discrete space setup
1. Discrete Ergodic Coverage
2. Maximizing Information Gathering

In [24]:
distribution_name = "one_peak"
time_steps = 100
K = 5
system1_name = "mm_1"
system2_name = "ig_1"
compute_mu = False
grid_length = 10

In [25]:
import numpy as np
U_shape = (1,1)
all_k = list(np.ndindex(*[K]*len(U_shape)))

# 0. Discrete Space Setup

## 0.a. Define probability distribution $\mu$

In [26]:
# defining probability distribution mu
from probability_distribution import *

mu = mu_gaussians([(np.array([0.2, 0.75]), 0.1)], U_shape)
mu = mu_normalize(mu, U_shape)

# calculating fourier coefficients of probability distribution mu
from fourier_functions import *
import pickle

ff = Fourier_Functions(mu, U_shape, K, compute_mu=compute_mu)
if compute_mu:
    mu_k = {}
    for k in ff:
        mu_k[k] = ff[k]['mu_k']

    with open(f'mu/{distribution_name}_{K}.pkl', 'wb') as handle:
        pickle.dump(mu_k, handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open(f'mu/{distribution_name}_{K}.pkl', 'rb') as handle:
        mu_k = pickle.load(handle)
    for k in ff:
        ff[k]['mu_k'] = mu_k[k]

original_mu = mu 


## 1. Generate Vanilla Ergodic Trajectory $x$

In [27]:
from ergodic_agents import *
from mm_agent import *

agent1 = MMAgent1(0, np.array([0.2, 0.3]), 0.5, all_k, U_shape, ff, eps=1e-5)
system1 = AgentSystem([agent1], mu, U_shape, ff, K)

In [28]:
t = 0
delta_t = 0.1
for i in range(time_steps):
    t = i * delta_t
    system1.evolve(t, delta_t)


In [ ]:
filename1 = f"optimality/system={system1_name}_mu={distribution_name}_K={K}_T={time_steps}"
description = f"{system1_name} on {distribution_name}"

system1.visualize_trajectory(filename1, description)
system1.visualize_ergodicity(filename1)

## 2. Maximize Information Gathering

In [ ]:
dx = 0.001
_mu_casadi = mu_gaussians_casadi([(np.array([0.2, 0.75]), 0.1)], U_shape)
total = mu_total(mu, U_shape)
mu_casadi = lambda x: _mu_casadi(x)/total

In [ ]:
mu_casadi(np.array([0.2, 0.35]))

DM(0.201897)

In [ ]:
from casadi_agent_info_gathering import *
agent2 = CasadiAgentInfoGathering(0, np.array([0.2, 0.3]), 0.5, all_k, U_shape, ff, eps=1e-5)
agent2.mu = mu_casadi #lambda x: casadi.norm_2(x) # need to do gaussian but for casadi
agent2.dx = dx
system2 = AgentSystem([agent2], mu_casadi, U_shape, ff, K)

In [ ]:
t = 0
delta_t = 0.1
for i in range(time_steps):
    t = i * delta_t
    system2.evolve(t, delta_t)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   4.23 s ( 10.31ms)   4.21 s ( 10.27ms)       410
       nlp_g  |  50.93ms (124.21us)  50.57ms (123.34us)       410
  nlp_grad_f  |   7.95 s ( 24.09ms)   7.91 s ( 23.97ms)       330
  nlp_hess_l  | 708.93 s (  2.16 s) 705.67 s (  2.15 s)       328
   nlp_jac_g  | 605.22ms (  1.83ms) 603.25ms (  1.83ms)       330
       total  | 735.04 s (735.04 s) 731.66 s (731.66 s)         1
[[ 5.55587563e-08  4.66472447e-08  3.82493786e-08  3.02292360e-08
   2.22584623e-08  1.33957547e-08 -2.75131780e-10 -5.04866149e-08
  -2.26963635e-02 

In [ ]:
filename2 = f"optimality/system={system2_name}_mu={distribution_name}_dx={dx}_T={time_steps}"
description = f"{system2_name} on {distribution_name}"

system2.visualize_trajectory(filename2, description)
# system2.visualize2d(filename=filename2, additional_title=description, plot_c_k=False)
system2.visualize_ergodicity(filename2)

## 3. Comparison

In [ ]:
results = {}

results['ergodic_trajectory'] = {}
results['ergodic_trajectory']['trajectory'] = np.array(agent1.x_log)
results['ergodic_trajectory']['ergodicity'] = agent1.e_log
results['ergodic_trajectory']['info gathered'] = [info_gathering_metric_noncasadi(mu, agent1.x_log, T+1, 1/dx) for T in range(time_steps)]

results['info_gather_trajectory'] = {}
results['info_gather_trajectory']['trajectory'] = np.array(agent2.x_log)
results['info_gather_trajectory']['ergodicity'] = np.array(agent2.e_log)
results['info_gather_trajectory']['info gathered'] = np.array([info_gathering_metric(mu_casadi, agent2.x_log, T+1, 1/dx) for T in range(time_steps)]).reshape(-1,)


In [ ]:
filename = f"optimality/system={system1_name}_{system2_name}_mu={distribution_name}_K={K}_dx={dx}_T={time_steps}"

with open(f'{filename}_results.pkl', 'wb') as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

def plot_trajectory_values(results, filename, trajectory_name):
    plt.figure()
    plt.plot(results[trajectory_name]['ergodicity'][1:], results[trajectory_name]['info gathered'], 'o')
    plt.xlabel("Ergodicity")
    plt.ylabel("Information Gathered")
    plt.savefig(f"{filename}_ergodicity_vs_info_gathered.pdf")
plot_trajectory_values(results, filename1, 'ergodic_trajectory')
plot_trajectory_values(results, filename2, 'info_gather_trajectory')

trajectories = [results['ergodic_trajectory']['trajectory'], results['info_gather_trajectory']['trajectory']]
visualize_trajectory(f"{filename}_ergodicity_vs_info_gathered", 
                     f"Ergodic vs Info Gathering Trajectory", 
                     U_shape, trajectories, mu)


In [ ]:
for trajectory in ['ergodic_trajectory', 'info_gather_trajectory']:
    print("Trajectory: ", trajectory)
    print("Final ergodicity: ", results[trajectory]['ergodicity'][-1])
    print("Final Info Gather: ", results[trajectory]['info gathered'][-1])